In [12]:
pip install opencv-python-headless

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture('video.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Show the frame
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [14]:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)


In [15]:
def region_of_interest(edges):
    height, width = edges.shape
    mask = np.zeros_like(edges)

    # Only focus on the bottom triangle of the video
    triangle = np.array([
        [(100, height), (width - 100, height), (width // 2, height // 2)]
    ], np.int32)

    cv2.fillPoly(mask, triangle, 255)
    masked_image = cv2.bitwise_and(edges, mask)
    return masked_image

def display_lines(frame, lines):
    line_image = np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 10)
    return line_image

# In your main loop, after converting to edges
masked_edges = region_of_interest(edges)
lines = cv2.HoughLinesP(masked_edges, 2, np.pi / 180, 100, np.array([]), minLineLength=40, maxLineGap=5)
line_image = display_lines(frame, lines)

# Combine the line image with the original frame
combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)


In [16]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use the codec compatible with QuickTime
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (width, height))

while cap.isOpened():
    # Your processing code
    out.write(combo_image)  # Write out the frame

    # Your display and break code

cap.release()
out.release()  # Don't forget to release the writer
cv2.destroyAllWindows()
